# Lanczos Method
Author: Micheal C. Chen

Email: muchuchen03@gmail.com

## Task:
In this demo I will demonstrate how to implement Lanczos method for exact diagonalization of spin system. And we will use Lanczos method to solve Heisenberg model.

## Bit Operations
The most significant part of Exact Diagonalization is to define proper bit operations.

In [2]:
def FlipBit(i, n): # Flip n-th bit
    return i^(1<<n)

def ReadBit(i, n): # Read n-th bit
    return (i&(1<<n))>>n

## Heisenberg Model
Consider a 1D chain Heisenberg model with Hamiltonian:
$$
H = J\sum_i \mathbf{S}_i \cdot \mathbf{S}_{i+1} = J \sum_{i=1}^{N}\left(S_i^{x}S_{i+1}^{x}+S_i^{y}S_{i+1}^{y}+S_i^{z}S_{i+1}^{z}\right) = J\sum_i \left(\cfrac{S_i^{+}S_{i+1}^{-}+S_i^{-}S_{i+1}^{+}}{2} + S_i^{z}S_{i+1}^{z}\right)
$$
In Lanczos method, we should avoid writing the full representation of the Hamiltonian.
* Off-diagonal term: $\cfrac{S_i^{+}S_{i+1}^{-}+S_i^{-}S_{i+1}^{+}}{2}$
* Diagonal term: $S_i^{z}S_{i+1}^{z}$

In [3]:
import numpy as np
import numpy.linalg as LA

In [4]:
def GetHopList(Ns):
    HopList = []
    for i in range(Ns-1):
        HopList.append([i, i+1]) # Ordinary Boundary Condition
    return HopList

def ApplyHeisenberg(Ns, state):
    """
    Apply Heisenberg model Hamiltonian for an given state.
    Args:
        Ns: Number of sites.
        state: Input state vector.
    Returns:
        result_state: Resulting state vector after applying the Hamiltonian.
    """
    dim = len(state)
    HopList = GetHopList(Ns)
    result_state = np.zeros(dim, dtype = complex) # initialize a zero vector, add dtype otherwise there will exist 'ComplexWarning'
    
    for basis in range(dim):
        old_config = basis
        for idx in range(len(HopList)): 
            Pos0 = HopList[idx][0]
            Pos1 = HopList[idx][1]

            if ReadBit(basis, Pos0) != ReadBit(basis, Pos1): # off-diagonal matrix operation results
                new_config = FlipBit(old_config, Pos0)
                new_config = FlipBit(new_config, Pos1)
                result_state[new_config] += 0.5 * J * state[old_config]

            result_state[old_config] += (ReadBit(old_config, Pos0)-0.5) * (ReadBit(old_config, Pos1)-0.5) * state[old_config] # diagonal matrix operation results

    return result_state


Then we set parameters and generate an initial random state as the first Lanczos eigenvector

In [34]:
Ns = 10 # Number of sites
J = 1.0 # Parameter

# Generate a random arbitrary state
dim = 2**Ns
init_state = np.random.rand(dim)+1j * np.random.rand(dim)
init_state = init_state/np.sqrt(np.conj(init_state).T @ init_state)

## Lanczos Iteration
The iterative procedure to construct Lanczos basis $\{|\phi_0\rangle, |\phi_1\rangle, \ldots, |\phi_n\rangle\}$ is :

$|\phi_{m+1}\rangle = \cfrac{1}{b_m}\left(H|\phi_m\rangle - a_m |\phi_m\rangle-b_{m-1}|\phi_{m-1}\rangle\right)$

where $b_m$ is a real number to ensure $\langle\phi_m|\phi_m\rangle=1$, and $a_m = \langle \phi_m|\phi_m\rangle$, $|\phi_{-1}\rangle=0$, $b_{-1}=0$.

We span the Hamiltonian in Lanczos basis and yield a tridiagonal matrix:
$$
H = \begin{pmatrix}
a_{0} & b_{0} & 0 & 0 & \cdots & 0 \\
b_{0} & a_{1} & b_{1} & 0 & \cdots & \vdots \\
0 & b_{1} & a_{2} & b_{2} & \cdots & 0 \\
\vdots & \ddots & \ddots & \ddots & \ddots & \vdots \\
0 & \cdots & 0 & b_{n-1} & a_{n}
\end{pmatrix}
$$

Finally we diagonalize the Lanczos matrix to find the ground state energy and low-lying excited states.

In [35]:
krylov_dim = 20
def LanczosIter(Ns, init_state):
    """
    Perform Lanczos iteration steps.
    Args:
        Ns: Number of sites
        init_state: A random initial state for Lanczos iteration.

    Returns:
        Lanczos basis: A list of Lanczos basis vectors.
        a_value: A list of diagonal elements of final tridiagonal matrix.
        b_value: A list of off-diagonal elements of final tridiagonal matrix.
    """
    lanczos_basis = []
    a_value = []
    b_value = []
    for i in range(krylov_dim):
        if i == 0: # init state is the first Lanczos basis
            lanczos_basis.append(init_state)
            a_value.append(np.inner(np.conj(init_state), ApplyHeisenberg(Ns, init_state)).real)

        if i == 1: #
            iter_basis = ApplyHeisenberg(Ns, lanczos_basis[i-1])- a_value[i-1] * lanczos_basis[i-1]
            b_value.append(np.sqrt(np.abs(np.inner(np.conj(iter_basis), iter_basis))))
            iter_basis = iter_basis/b_value[i-1]
            a_value.append(np.inner(np.conj(iter_basis), ApplyHeisenberg(Ns, iter_basis)).real)
            lanczos_basis.append(iter_basis)

        if i > 1:
            iter_basis = ApplyHeisenberg(Ns, lanczos_basis[i-1])- a_value[i-1] * lanczos_basis[i-1] - b_value[i-2]* lanczos_basis[i-2]
            b_value.append(np.sqrt(np.abs(np.inner(np.conj(iter_basis), iter_basis))))
            iter_basis = iter_basis/b_value[i-1]
            a_value.append(np.inner(np.conj(iter_basis), ApplyHeisenberg(Ns, iter_basis)).real)
            lanczos_basis.append(iter_basis)

    return lanczos_basis, a_value, b_value

def LanczosMat(krylov_dim, a_value, b_value):
    """
    Construct Lanczos matrix.
    Args:
        krylov_dim: Dimension of Krylov subspace.
        a_value: A list of diagonal elements of final tridiagonal matrix.
        b_value: A list of off-diagonal elements of final tridiagonal matrix.
    Returns:
        lanczos_mat: Lanczos matrix.
    """
    lanczos_mat = np.zeros((krylov_dim, krylov_dim), dtype = complex)
    for i0 in range(krylov_dim): # diagonal-term
        
        lanczos_mat[i0][i0] = a_value[i0]
    for i0 in range(krylov_dim-1):
        lanczos_mat[i0][i0+1] = b_value[i0]
        lanczos_mat[i0+1][i0] = b_value[i0]
    
    return lanczos_mat

In [36]:
lanczos_basis, a_value, b_value = LanczosIter(Ns, init_state)
lanczos_mat = LanczosMat(krylov_dim, a_value, b_value)
eigvals, eigvecs_lanc = LA.eigh(lanczos_mat)
print("Eigenvalues:", np.sort(eigvals))

Eigenvalues: [-4.2580352  -3.9306715  -3.52633143 -3.14900567 -2.90411859 -2.57088094
 -2.18188946 -1.75951332 -1.34197492 -0.90037035 -0.43465121 -0.0214522
  0.39731753  0.78688084  1.17729837  1.48617444  1.77192234  2.00559997
  2.15364073  2.2499879 ]


After solving the eigenvalues, we can quickly calculate the eigenstates of origin Hamiltonian.

In [37]:
dim_H = lanczos_basis[0].shape[0]
eigvecs_full = np.zeros((dim_H, krylov_dim), dtype = complex)
for i in range(krylov_dim):
    psi_k = np.zeros(dim_H, dtype = complex)
    for j in range(krylov_dim):
        psi_k += eigvecs_lanc[j][i] * lanczos_basis[j]
    eigvecs_full[:, i] = psi_k/np.sqrt(np.inner(np.conj(psi_k), psi_k))